<a href="https://colab.research.google.com/github/armandossrecife/piloto/blob/main/notebooks/analyse30repoPOO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!rm -rf repositorio*

# Classes

## Dependências

In [13]:
!pip3 install gitpython

In [16]:
import requests
import pandas as pd
import os
import threading
import datetime
from git import Repo

class GitHubRepoChecker:
    def __init__(self, url):
        self.url = url
        self.df_repositorios = None
        self.lista_urls_validas = []
        self.lista_urls_invalidas = []

    def check_url(self, url):
        try:
            response = requests.get(url)
            if response.status_code == 200:
                return True
            else:
                return False
        except requests.exceptions.RequestException:
            return False

    def load_data(self):
        self.df_repositorios = pd.read_csv(filepath_or_buffer=self.url, sep=';')

    def validate_urls(self):
        if self.df_repositorios is None:
            raise ValueError("Os dados não foram carregados. Chame o método 'load_data()' primeiro.")

        lista_url_repositorios = self.df_repositorios['Git Repository'].tolist()

        for url in lista_url_repositorios:
            url_valida = self.check_url(url)
            if url_valida:
                self.lista_urls_validas.append(url)
            else:
                self.lista_urls_invalidas.append(url)

    def get_invalid_urls(self):
        return self.lista_urls_invalidas

    def get_valid_urls(self):
        return self.lista_urls_validas


class RepoCloner:
    def __init__(self, qtd_repositorios, lista_urls):
        self.qtd_repositorios = qtd_repositorios
        self.lista_urls = lista_urls
        self.lista_threads = []
        self.lista_tempo_inicio = []
        self.lista_tempo_conclusao = []

    def my_clone(self, git_url, repo_dir):
        try:
            Repo.clone_from(git_url, repo_dir)
            tempo = datetime.datetime.now()
            print(f'Clonagem do repo {git_url} concluída com sucesso! -> instante: {tempo}')
            elemento = (repo_dir, tempo)
            self.lista_tempo_conclusao.append(elemento)
        except Exception as ex:
            print(f"Erro ao clonar o repo: {git_url} - {str(ex)}")

    def clona_repositorios(self):
        for i in range(self.qtd_repositorios):
            nome = 'repositorio'+str(i+1)
            tdi = threading.Thread(target=self.my_clone, args=(self.lista_urls[i], nome))
            tdi.name = 'Thread-'+str(i+1)
            inicio_thread = datetime.datetime.now()
            print(f"Thread {i+1} iniciada em {inicio_thread} - clonando {nome}:{self.lista_urls[i]}...")
            self.lista_threads.append(tdi)
            elemento = (nome, inicio_thread)
            self.lista_tempo_inicio.append(elemento)
            tdi.start()

        for each in self.lista_threads:
            each.join()

    def mostra_tempo_clonagens(self):
        for t1 in self.lista_tempo_inicio:
            for t2 in self.lista_tempo_conclusao:
                if t1[0] == t2[0]:
                    print(f"Tempo de download do {t1[0]} é: {t2[1]-t1[1]} s")


class RepositorioChecker:
    def __init__(self, url_repositorios_src, url_complemento):
        self.df_repositorios_src = pd.read_csv(url_repositorios_src)
        self.df_complemento = pd.read_csv(url_complemento)
        self.df_repositorios = None

    def merge_dataframes(self):
        self.df_complemento['nome'] = self.df_repositorios_src['projeto'].to_numpy()
        self.df_complemento['source_java'] = self.df_repositorios_src['source'].to_numpy()
        self.df_repositorios = self.df_complemento.copy()
        self.df_repositorios['source_java'] = self.df_repositorios['source_java'].str.replace('.', '/')
        self.df_repositorios = self.df_repositorios[['nome', 'repositorio', 'source_java', 'url']]
        self.df_repositorios.to_csv('l31repositoriossrc.csv', index=False)

    def check_sub_path(self, sub_path, repositorio):
        current_path = os.getcwd()
        abs_sub_path = os.path.join(current_path, repositorio, sub_path)
        print(f"Analisa {sub_path} em {repositorio} - {os.path.exists(abs_sub_path)}")
        return os.path.exists(abs_sub_path)

    def check_repositorios(self):
        if self.df_repositorios is None:
            raise ValueError("Os dados não foram carregados. Chame o método 'merge_dataframes()' primeiro.")

        lista_nomes_repositorio = self.df_repositorios['nome'].tolist()
        lista_repositorios = self.df_repositorios['repositorio'].tolist()
        lista_src = self.df_repositorios['source_java'].tolist()
        lista_checa = []
        lista_urls = self.df_repositorios['url'].to_list()

        for i, repositorio in enumerate(lista_repositorios):
            resultado = self.check_sub_path(sub_path=lista_src[i], repositorio=repositorio)
            lista_checa.append(resultado)

        dict_temp = {
            'nome': lista_nomes_repositorio,
            'repositorio': lista_repositorios,
            'url': lista_urls,
            'src': lista_src,
            'checa': lista_checa
        }

        df_checa_repositorios = pd.DataFrame(dict_temp)
        return df_checa_repositorios

# 1. Carrega os dados dos repositórios

In [17]:
# URLs dos repositórios
URL = 'https://raw.githubusercontent.com/armandossrecife/teste/main/listaderepositorios.csv'

# Verifica a validade das URLs dos repositórios
checker = GitHubRepoChecker(URL)
checker.load_data()
checker.validate_urls()

invalid_urls = checker.get_invalid_urls()
valid_urls = checker.get_valid_urls()

print(f'URLs inválidas:')
for url in invalid_urls:
    print(url)

print(f'URLs válidas:')
for url in valid_urls:
    print(url)

URLs inválidas:
URLs válidas:
https://github.com/apache/accumulo.git
https://github.com/apache/calcite.git
https://github.com/apache/cassandra.git
https://github.com/apache/chukwa.git
https://github.com/apache/jackrabbit.git
https://github.com/FasterXML/jackson.git
https://github.com/apache/jspwiki.git
https://github.com/square/retrofit.git
https://github.com/spring-projects/spring-boot.git
https://github.com/apache/struts.git
https://github.com/elastic/elasticsearch.git
https://github.com/apache/jena.git
https://github.com/apache/lucene-solr.git
https://github.com/apache/tika.git
https://github.com/apache/ant-ivy.git
https://github.com/jenkinsci/jenkins.git
https://github.com/spearce/jgit.git
https://github.com/SeleniumHQ/selenium.git
https://github.com/cbeust/testng.git
https://github.com/apache/pdfbox.git
https://github.com/apache/poi.git
https://github.com/apache/xerces2-j.git
https://github.com/apache/druid.git
https://github.com/pgjdbc/pgjdbc.git
https://github.com/apache/activem

## Dados dos 30 repositórios válidos

In [18]:
df_repositorios = checker.df_repositorios
df_repositorios

,Category,Project,Git Repository,Description
0,Data storage and management,accumulo,https://github.com/apache/accumulo.git,Data Storage System
1,Data storage and management,calcite,https://github.com/apache/calcite.git,Dynamic Data Managment
2,Data storage and management,cassandra,https://github.com/apache/cassandra.git,Distributed NoSQL database
3,Data storage and management,chukwa,https://github.com/apache/chukwa.git,Data collection
4,Data storage and management,jackrabbit,https://github.com/apache/jackrabbit.git,Content Repository
5,Data storage and management,jackson,https://github.com/FasterXML/jackson.git,Data biding library
6,Web engines and web tools,jspwiki,https://github.com/apache/jspwiki.git,Wiki Engine
7,Web engines and web tools,retrofit,https://github.com/square/retrofit.git,Android HTTP client
8,Web engines and web tools,spring-boot,https://github.com/spring-projects/spring-boot...,Spring-based project manager
9,Web engines and web tools,struts,https://github.com/apache/struts.git,Web apps


## 2. Clona os respositórios

In [19]:
# Clona os repositórios
lista_url_repositorios = checker.get_valid_urls()
qtd_repositorios = len(lista_url_repositorios)

print(f'Clonando os {qtd_repositorios} repositórios. Aguarde...')
cloner = RepoCloner(qtd_repositorios, lista_url_repositorios)
cloner.clona_repositorios()
cloner.mostra_tempo_clonagens()

Clonando os 30 repositórios. Aguarde...
Thread 1 iniciada em 2023-07-15 02:54:38.705156 - clonando repositorio1:https://github.com/apache/accumulo.git...
Thread 2 iniciada em 2023-07-15 02:54:38.706071 - clonando repositorio2:https://github.com/apache/calcite.git...
Thread 3 iniciada em 2023-07-15 02:54:38.713225 - clonando repositorio3:https://github.com/apache/cassandra.git...
Thread 4 iniciada em 2023-07-15 02:54:38.717553 - clonando repositorio4:https://github.com/apache/chukwa.git...
Thread 5 iniciada em 2023-07-15 02:54:38.721324 - clonando repositorio5:https://github.com/apache/jackrabbit.git...
Thread 6 iniciada em 2023-07-15 02:54:38.727437 - clonando repositorio6:https://github.com/FasterXML/jackson.git...
Thread 7 iniciada em 2023-07-15 02:54:38.731676 - clonando repositorio7:https://github.com/apache/jspwiki.git...
Thread 8 iniciada em 2023-07-15 02:54:38.738487 - clonando repositorio8:https://github.com/square/retrofit.git...
Thread 9 iniciada em 2023-07-15 02:54:38.742306

# 3. Recupera informação do diretório src/main

In [11]:
URL_DATASET_SRC_REPOSITORIOS = 'https://raw.githubusercontent.com/armandossrecife/teste/main/l31repositorios.csv'
URL_DATASET_CLONAGEM_REPOSITORIOS = 'https://raw.githubusercontent.com/armandossrecife/teste/main/l31repositoriosurl.csv'

# Verifica a existência de subdiretórios nos repositórios
repositorio_checker = RepositorioChecker(URL_DATASET_SRC_REPOSITORIOS, URL_DATASET_CLONAGEM_REPOSITORIOS)

repositorio_checker.merge_dataframes()
df_checa_repositorios = repositorio_checker.check_repositorios()

# Exibe o resultado da verificação
df_checa_repositorios

Analisa core/src/main/java em repositorio1 - True
Analisa core/src/main/java em repositorio2 - True
Analisa src/java em repositorio3 - True
Analisa core/src/main/java em repositorio4 - True
Analisa jackrabbit-core/src/main/java em repositorio5 - True
Analisa nao tem o src the Java JSON library em repositorio6 - False
Analisa jspwiki-api/src/main/java em repositorio7 - True
Analisa retrofit/src/main/java em repositorio8 - True
Analisa spring-boot-project/spring-boot/src/main/java em repositorio9 - True
Analisa core/src/main/java em repositorio10 - True
Analisa server/src/main/java em repositorio11 - True
Analisa jena-core/src/main/java em repositorio12 - True
Analisa nao tem o src lucene-solr em repositorio13 - False
Analisa tika-core/src/main/java em repositorio14 - True
Analisa src/java em repositorio15 - True
Analisa core/src/main/java em repositorio16 - True
Analisa org/eclipse/jgit/src/org/eclipse/jgit em repositorio17 - False
Analisa java/src em repositorio18 - True
Analisa testng

<ipython-input-10-29ea73180399>:13: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  self.df_repositorios['source_java'] = self.df_repositorios['source_java'].str.replace('.', '/')


,nome,repositorio,url,src,checa
0,accumulo,repositorio1,https://github.com/apache/accumulo.git,core/src/main/java,True
1,calcite,repositorio2,https://github.com/apache/calcite.git,core/src/main/java,True
2,cassandra,repositorio3,https://github.com/apache/cassandra.git,src/java,True
3,chukwa,repositorio4,https://github.com/apache/chukwa.git,core/src/main/java,True
4,jackrabbit,repositorio5,https://github.com/apache/jackrabbit.git,jackrabbit-core/src/main/java,True
5,jackson,repositorio6,https://github.com/FasterXML/jackson.git,nao tem o src the Java JSON library,False
6,jspwiki,repositorio7,https://github.com/apache/jspwiki.git,jspwiki-api/src/main/java,True
7,retrofit,repositorio8,https://github.com/square/retrofit.git,retrofit/src/main/java,True
8,spring-boot,repositorio9,https://github.com/spring-projects/spring-boot...,spring-boot-project/spring-boot/src/main/java,True
9,struts,repositorio10,https://github.com/apache/struts.git,core/src/main/java,True
